In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math
import time
import matplotlib.pyplot as plt

#### Encoder-Decoder Architecture

In [2]:
class EncoderDecoder(nn.Module):
    """A standard Encoder-Decoder architecture.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        """Take in and process masked src and target sequences.
           
           Args:
              src: encoder input. [batch_size, encoder_seq_len]
              tgt: decoder input. [batch_size, decoder_seq_len]
              src_mask: encoder input mask. 
                  [batch_size, 1, encoder_seq_len]
              tgt_mask: decoder input mask. 
                  [batch_size, decoder_seq_len, decoder_seq_len]
        """
        return self.decode(self.encode(src, src_mask), src_mask, 
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [3]:
class Generator(nn.Module):
    """Define standard linear + softmax generation step.
    """
    def __init__(self, d_model, vocab_size):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

#### Encoder Stacks

In [4]:
def clones(module, N):
    """Clone N identical layers.
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [5]:
class Encoder(nn.Module):
    """Define the encoer which is a stack of N identical blocks.
    """
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        """Args:
               x: [batch_size, encoder_seq_len, d_model]
               mask: [batch_size, 1, encoder_seq_len]
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [6]:
class LayerNorm(nn.Module):
    """Construct a layer normalization module.
    """
    def __init__(self, layer_size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(layer_size))
        self.b_2 = nn.Parameter(torch.zeros(layer_size))
        self.eps = eps
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [7]:
class SublayerConnection(nn.Module):
    """A residual connection followed by a layer norm.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        """Apply residual connection to any sublayer with the same size.
        """
        return x + self.dropout(sublayer(self.norm(x)))

In [8]:
class EncoderLayer(nn.Module):
    """Encoder is made up of self-attention and feed forward network.
    """
    def __init__(self, size, self_attn,  feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayers = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        
    def forward(self, x, mask):
        """Args:
               x: [batch_size, encoder_seq_len, d_model]
               mask: [batch_size, 1, encoder_seq_len]
        """
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayers[1](x, self.feed_forward)

#### Decoder Stacks

In [9]:
class Decoder(nn.Module):
    """Generic N layer deocder with masking.
    """
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        """Args:
              x: dncoder embeddins. [batch_size, decoder_seq_len, d_model]
              memory: [batch_size, encoder_seq_len, d_model]
              src_mask: encoder input mask. 
                  [batch_size, 1, encoder_seq_len]
              tgt_mask: decoder input mask. 
                  [batch_size, decoder_seq_len, decoder_seq_len]
        """
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [10]:
class DecoderLayer(nn.Module):
    """Decoder is made of self-attention, src-attetion and feed forward.
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayers = clones(SublayerConnection(size, dropout), 3)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        """Args:
              x: dncoder embeddins. [batch_size, decoder_seq_len, d_model]
              memory: [batch_size, encoder_seq_len, d_model]
              src_mask: encoder input mask. 
                  [batch_size, 1, encoder_seq_len]
              tgt_mask: decoder input mask. 
                  [batch_size, decoder_seq_len, decoder_seq_len]
        """
        m = memory
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayers[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayers[2](x, self.feed_forward)

In [11]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [12]:
# subsequent_mask(10)

In [13]:
def attention(query, key, value, mask=None, dropout=None):
    """Scaled Dot Product Attention.
    
       Args:
           query: [batch_size, head_num, seq_len, d_k]
           key:   [batch_size, head_num, seq_len, d_k]
           value: [batch_size, head_num, seq_len, d_k]
           mask:  [batch_size, 1, 1, encoder_seq_len] or 
                  [batch_size, 1, decoder_seq_len, decoder_seq_len]
    """
    d_k = query.size(-1)
    # scores: [... , seq_len, seq_len]
    scores = torch.matmul(query, key.transpose(-2, -1)) \
        / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [14]:
class MultiHeadedAttention(nn.Module):
    """Inplement multi-head attention.
    """
    def __init__(self, head_num, d_model, dropout=0.1):
        """Take in model size and number of heads.
        """
        super(MultiHeadedAttention, self).__init__()
        assert d_model % head_num == 0
        # Notice here assuming that d_model == head_num * d_v and
        # d_v == d_k
        self.d_k = d_model // head_num
        self.head_num = head_num
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        """Implements the multi-head attention.
            
           Args:
               query: [batch_size, seq_len, d_model]
               key:   [batch_size, seq_len, d_model]
               value: [batch_size, seq_len, d_model]
               mask:  [batch_size, 1, 1, encoder_seq_len] or 
                      [batch_size, 1, decoder_seq_len, decoder_seq_len]
        """
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0) 
        
        # query: [batch_size, head_num, seq_len, d_k]
        query, key, value = [l(x).view(nbatches, -1, self.head_num, self.d_k).transpose(1, 2) \
                             for l, x in zip(self.linears, (query, key, value))]
        
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # x: [batch_size, seq_len, d_model]
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.head_num * self.d_k)
        return self.linears[-1](x)

#### Position-wise Feed-forward Networks

In [15]:
class PositionwiseFeedForward(nn.Module):
    """Implements FNN equation.
    """
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

#### Embeddings and Softmax

In [16]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab_size):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [17]:
class PositionalEncoding(nn.Module):
    """Implement the PE function.
    """
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space
        pe = torch.zeros(max_len, d_model)
        # position: [max_len, 1]
        position = torch.arange(0., max_len).unsqueeze(1)
        # div_term: [d_model/2]
        div_term = torch.exp(torch.arange(0., d_model, 2) * 
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe.detach()[:, :x.size(1)]
        return self.dropout(x)

In [18]:
# plt.figure(figsize=(15, 5))
# pe = PositionalEncoding(20, 0)
# y = pe.forward(Variable(torch.zeros(1, 100, 20)))
# plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())
# plt.legend(["dim %d"%p for p in [4,5,6,7]])

#### Full Model

In [19]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, 
               d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model 

In [20]:
tmp_model = make_model(10, 10, 2)

#### Batches and Training

In [21]:
class Batch:
    """Object for holding a batch of data with mask during training.
    """
    def __init__(self, src, trg=None, pad=0):
        self.src = src # [batch_size, seq_len]
        self.src_mask = (src != pad).unsqueeze(-2) # [batch_size, 1, seq_len]
        
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]      
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).sum().item()
            
    @staticmethod
    def make_std_mask(tgt, pad):
        """Create a mask to hide padding and future words.
        """
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)
        return tgt_mask

#### Training Loop

In [22]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        # out: [batch_size, seq_len, d_model]
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

#### Training Data and Batching

In [23]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch, len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch, len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

#### Optimizer

In [24]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) * \
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [25]:
# # Three settings of the lrate hyperparameters.
# opts = [NoamOpt(512, 1, 4000, None), 
#         NoamOpt(512, 1, 8000, None),
#         NoamOpt(256, 1, 4000, None)]
# plt.plot(np.arange(1, 20000), [[opt.rate(i) for opt in opts] for i in range(1, 20000)])
# plt.legend(["512:4000", "512:8000", "256:4000"])

#### Label Smoothing

In [26]:
class LabelSmoothing(nn.Module):
    """Implement label smoothing.
    """
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size # vocal size
        self.true_dist = None
        
    def forward(self, x, target):
        """Compute loss with label smoothing.
        
           Args:
               x: reshaped softmax output. 
                   [batch_size * target_seq_len, vocab]
               target: reshaped target labels.
                   [batch_size * target_seq_len]
        """
        assert x.size(1) == self.size
        true_dist = x.clone().detach()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.detach().unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        # mask: [non_zero_num, 1]
        mask = torch.nonzero(target.detach() == self.padding_idx)
        if mask.sum() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

In [27]:
# # Example of label smoothing.
# crit = LabelSmoothing(5, 0, 0.4)
# predict = torch.FloatTensor([[0, 0.2, 0.7, 0.1, 0],
#                              [0, 0.2, 0.7, 0.1, 0], 
#                              [0, 0.2, 0.7, 0.1, 0]])
# v = crit(Variable(predict.log()), 
#          Variable(torch.LongTensor([2, 1, 0])))

# # Show the target distributions expected by the system.
# plt.imshow(crit.true_dist)

In [28]:
# crit = LabelSmoothing(5, 0, 0.1)
# def loss(x):
#     d = x + 3 * 1
#     predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d],
#                                  ])
#     #print(predict)
#     return crit(Variable(predict.log()),
#                  Variable(torch.LongTensor([1]))).item()
# plt.plot(np.arange(1, 100), [loss(x) for x in range(1, 100)])

### A First Example

#### Synthetic Data

In [29]:
def data_gen(V, batch, nbatches):
    """Generate random data for a src-tgt copy task.
    """
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10), dtype=np.int64))
        data[:, 0] = 1
        src = data
        tgt = data.clone()
        yield Batch(src, tgt, 0)

#### Loss Computation

In [30]:
class SimpleLossCompute:
    """A simple loss compute and train function.
    """
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion # label smoothing
        self.optimizer = opt
    
    def __call__(self, x, y, norm):
        """Comput loss, back propagate and updata parameters.
        
           x: last layer output, [batch_size, target_seq_len, d_model]
           y: target, [batch_size, target_seq_len]
           norm: scalar
        """
        x = self.generator(x) # [batch_size, target_seq_len, vocab]
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)),
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.optimizer is not None:
            self.optimizer.step()
            self.optimizer.optimizer.zero_grad()
        return loss.item() * norm

#### Greedy Decoding

In [31]:
# Train the simple copy task.
V = 17
model = make_model(V, V, N=2) # define the model
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0) # define criterion
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400, # define optimizer
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

print('start training...')
for epoch in range(10):
    model.train()
    run_epoch(data_gen(V, 30, 20), model, 
              SimpleLossCompute(model.generator, criterion, model_opt))
    model.eval()
    print(run_epoch(data_gen(V, 30, 5), model, 
                    SimpleLossCompute(model.generator, criterion, None)))

C:\Users\MengCao\Anaconda3\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


start training...
Epoch Step: 1 Loss: 3.607897 Tokens per Sec: 439.840720
Epoch Step: 1 Loss: 2.282511 Tokens per Sec: 716.195140
2.259409761428833
Epoch Step: 1 Loss: 2.346815 Tokens per Sec: 536.613210
Epoch Step: 1 Loss: 1.877462 Tokens per Sec: 730.693295
1.857159423828125
Epoch Step: 1 Loss: 2.025092 Tokens per Sec: 543.073308
Epoch Step: 1 Loss: 1.276650 Tokens per Sec: 773.489744
1.3382220983505249
Epoch Step: 1 Loss: 1.482565 Tokens per Sec: 546.912451
Epoch Step: 1 Loss: 0.972336 Tokens per Sec: 758.323730
1.0542756557464599
Epoch Step: 1 Loss: 1.266172 Tokens per Sec: 467.164296
Epoch Step: 1 Loss: 0.879321 Tokens per Sec: 780.178920
0.8990882158279419
Epoch Step: 1 Loss: 0.965779 Tokens per Sec: 573.562894
Epoch Step: 1 Loss: 0.550067 Tokens per Sec: 825.371178
0.5777664184570312
Epoch Step: 1 Loss: 0.698464 Tokens per Sec: 559.342458
Epoch Step: 1 Loss: 0.343054 Tokens per Sec: 797.413884
0.27760822474956515
Epoch Step: 1 Loss: 0.456689 Tokens per Sec: 549.131364
Epoch Step

In [32]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    """Implement greedy decode method.
    """
    memory = model.encode(src, src_mask)
    batch_size = src.shape[0]
    decode_seq = torch.ones(batch_size, 1).fill_(start_symbol).type_as(src.data)
    
    # start decoding
    with torch.no_grad():
        for i in range(max_len - 1):
            tgt_mask = subsequent_mask(decode_seq.shape[1])
            # output: [batch_size, seq_len, d_model]
            output = model.decode(memory, src_mask, decode_seq, tgt_mask)
            
            # prob: [batch_size, vocab_size]
            prob = model.generator(output[:, -1]) # the last word
            # next_word: [batch_size]
            _, next_word = torch.max(prob, dim=1)
            next_word = next_word.unsqueeze(1)
            # decode_seq: [batch_size, seq_len]
            decode_seq = torch.cat([decode_seq, next_word], dim=1)
    return decode_seq

# def greedy_decode(model, src, src_mask, max_len, start_symbol):
#     memory = model.encode(src, src_mask)
#     ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
#     for i in range(max_len-1):
#         out = model.decode(memory, src_mask, 
#                            ys, 
#                            subsequent_mask(ys.size(1)).type_as(src.data))
#         prob = model.generator(out[:, -1])
#         _, next_word = torch.max(prob, dim = 1)
#         next_word = next_word.data[0]
#         ys = torch.cat([ys, 
#                         torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
#     return ys

In [37]:
model.eval()
seq_len = 15
src = torch.LongTensor([range(1, seq_len + 1)]) # [2, 10]
src_mask = torch.ones(1, 1, seq_len)
print(greedy_decode(model, src, src_mask, max_len=seq_len*2, start_symbol=1))

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12,  9, 10,  9, 10,  8,  9, 15,
          9, 14,  9, 11,  8,  9, 15,  9, 14,  9, 12,  9]])


### A Real Word Example

#### Data Loading

In [ ]:
# to be continued...